In [1]:
from netCDF4 import Dataset
import wrf
import xarray as xr
import numpy as np

import cartopy.crs as crs


from matplotlib.cm import get_cmap
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cartopy.feature as cfe

import pandas as pd


import matplotlib.pyplot as plt



from cartopy.feature import NaturalEarthFeature

import cartopy.io.shapereader as shpr

import cartopy.io.img_tiles as cimgt

from wrf import (to_np, getvar, smooth2d, get_cartopy, cartopy_xlim,
                 cartopy_ylim, latlon_coords)

from matplotlib.cm import get_cmap
from wrf import getvar, interplevel, to_np, get_basemap, latlon_coords
import imageio

In [2]:
stamen_terrain = cimgt.Stamen('terrain-background')

states = NaturalEarthFeature(category="cultural", scale="10m",
                                 facecolor="none",
                                 name="admin_1_states_provinces_shp")
stt = NaturalEarthFeature(category='cultural', 
    name='admin_0_boundary_lines_land',
    scale='10m',facecolor='none')
stt_prv = states

roads = NaturalEarthFeature(category='cultural',
                                     name='roads',
                                     scale='10m',
                                     facecolor='none')

# Temperatura
## Función para hacer las animaciones de la temperatura a dos metros

In [12]:
# Abrir el dataset
def gif_temperatura(fichero,dominio):
    ncfile = Dataset(fichero)
    for i in range(0,169):
        # Leer la temperatura a 2 metros en eltiempo i 
        T2= getvar(ncfile, "T2",timeidx=i)

        #Calcular el día
        dia = int(i/24)
        hora =i- dia*24
        # Para leer la latitud y la longitud
        lats, lons = latlon_coords(T2)

        #Para leer la proyección en la que se ha hecho
        cart_proj = get_cartopy(T2)

        # Crear la figura para mostrarlo
        fig = plt.figure(figsize=(12,6))

        #Poner los GeoAxes en la proyección usada por el modelo 
        ax = plt.axes(projection=cart_proj)

        #Para mostrar las lineas de costa
        ax.coastlines('50m', linewidth=0.8)

        #Para hacer el contorno de los puntos que tienen misma temperatura
        #plt.contour(to_np(lons), to_np(lats), to_np(T2),  colors="gray",levels=   [j for j in range(270,302, 1)], transform=crs.PlateCarree(),linewidth=0.1)
                 
               
        #Para rellenar lo puntos que tienen la misma temperatura
        T2_contorno = plt.contourf(to_np(lons), to_np(lats), to_np(T2),
                 transform=crs.PlateCarree(),
                levels = [j for j in range(270,310, 1)],
                 cmap=get_cmap("jet"))


        # Add a color bar
        plt.colorbar(T2_contorno, ax=ax, shrink=.98)

        # Set the map bounds
        ax.set_xlim(cartopy_xlim(T2))
        ax.set_ylim(cartopy_ylim(T2))

        # Add the gridlines
        ax.gridlines(color="gray", linestyle="dotted")

        ax.add_feature(states, linewidth=.9, edgecolor="black")

        #Mostrar las carreteras 
        ax.add_feature(roads,linewidth=.4, edgecolor="black")

        plt.title("Temperatura a 2 metros (K) \n Dia {} Hora {}:00:00".format(dia+11,hora))
        plt.savefig('figs/d{}/temperatura/d{}_temperatura_{}.png'.format(dominio, dominio,i))
        #Para que no se muestren las imágenes
        plt.close(fig)
            
    with imageio.get_writer('figs/d{}/d{}_temperatura.gif'.format(dominio,dominio), mode='I') as writer:
        for i in range(0,169):
            for j in range(0,4):
                filename = 'figs/d{}/temperatura/d{}_temperatura_{}.png'.format(dominio,dominio,i)
                image = imageio.imread(filename)
                writer.append_data(image)

## Dominio 1

In [13]:
gif_temperatura('wrfout_d01_2021-05-11_00:00:00',"01")

<img src="figs/d01/d01_temperatura.gif" />

## Dominio 2

In [21]:
gif_temperatura('wrfout_d02_2021-05-11_00:00:00',"02")

<img src="figs/d02/d02_temperatura.gif" />

## Dominio 3


In [10]:
gif_temperatura('wrfout_d03_2021-05-11_00:00:00',"03")

<img src="figs/d03/d03_temperatura.gif" />

# Scrip para mostrar los 3 dominios a la vez

In [4]:
# Abrir el dataset
def gif_temperatura_3_doms(fichero):
    ncfile_01 = Dataset(fichero[0])
    ncfile_02 = Dataset(fichero[1])
    ncfile_03 = Dataset(fichero[2])

    for i in range(0,169):
        # Leer la presión a nivel del mar en eltiempo t 
        T_01 = getvar(ncfile_01, "T2",timeidx=i)
        T_02 = getvar(ncfile_02, "T2",timeidx=i)
        T_03 = getvar(ncfile_03, "T2",timeidx=i)
    
        
        T_03 = smooth2d(T_03, 3, cenweight=4)
        T_02 = smooth2d(T_02, 3, cenweight=4)
        
        
        niveles = [j for j in range(270,302, 1)]
        #Para leer la proyección en la que se ha hecho
        cart_proj = get_cartopy(T_03)

        # Crear la figura para mostrarlo
        fig = plt.figure(figsize=(15, 6))
        
        
        #Ciudad de Madrid
        # Para leer la latitud y la longitud
        lats, lons = latlon_coords(T_03)
        
        ax = fig.add_subplot(1, 5, (1,4), projection=cart_proj)
        # Set the map bounds
        ax.set_xlim(cartopy_xlim(T_03))
        ax.set_ylim(cartopy_ylim(T_03))
        
        ax.set_title('Temperatura a 2 metro(K) en Madrid \n Día {} Hora {}:00:00'.format(int(i*3/24)+11,(i*3-int(i*3/24)*24)), fontsize=16)
        ax.coastlines('50m', linewidth=0.8)
    
        #Para hacer el contorno de los puntos que tienen misma presión
        plt.contour(to_np(lons), to_np(lats), to_np(T_03),levels=niveles, 
                    colors="gray",
                    transform=crs.PlateCarree(),linewidth=0.1)
        #Para rellenar lo puntos que tienen la misma presión
        contorno_slp= plt.contourf(to_np(lons), to_np(lats), to_np(T_03), 10,
                     transform=crs.PlateCarree(),
                    levels=niveles,
                     cmap=get_cmap("jet"))

        # Add a color bar
        plt.colorbar(contorno_slp,ax=ax, shrink=.98)
        ax.add_feature(stt, linewidth=0.2, edgecolor='black')
        
        #Mostrar las carreteras 
        ax.add_feature(roads,linewidth=.4, edgecolor="black")

        # Add the gridlines
        ax.gridlines(color="gray", linestyle="dotted")

        
        
        
        # Comunidad de Madrid
        lats, lons = latlon_coords(T_02)
        
        ax_2 = fig.add_subplot(2, 5, 5,projection=cart_proj)
        
        ax_2.set_xlim(cartopy_xlim(T_02))
        ax_2.set_ylim(cartopy_ylim(T_02))
        
        ax_2.set_title('Comunidad de Madrid', fontsize=8)
        
        ax_2.coastlines('50m', linewidth=0.8)

        
        #plt.contour(to_np(lons), to_np(lats), to_np(T_02),levels=niveles,  colors="gray",transform=crs.PlateCarree(),linewidth=0.1)
                  
        
                    
        #Para rellenar lo puntos que tienen la misma presión
        contorno_slp= plt.contourf(to_np(lons), to_np(lats), to_np(T_02),
                     transform=crs.PlateCarree(),
                    levels=niveles,
                     cmap=get_cmap("jet"))
        
        ax_2.add_feature(stt,     linewidth=0.5, edgecolor='black')
        ax_2.add_feature(stt_prv, linewidth=0.2, edgecolor='black')
        ax_2.add_feature(roads,linewidth=.4, edgecolor="black")
        
        
        
        
        #España
        lats, lons = latlon_coords(T_01)
        
        ax_1 = fig.add_subplot(2, 5, 10, projection=cart_proj)
        
        ax_1.set_title('España', fontsize=8)
        ax_1.coastlines('50m', linewidth=0.8)
        
        #plt.contour(to_np(lons), to_np(lats), to_np(T_01),levels=niveles, colors="gray",transform=crs.PlateCarree(),linewidth=0.1)
                    
        #Para rellenar lo puntos que tienen la misma presión
        contorno_slp= plt.contourf(to_np(lons), to_np(lats), to_np(T_01),
                     transform=crs.PlateCarree(),
                    levels=niveles,
                     cmap=get_cmap("jet"))
        
        ax_1.add_feature(stt,     linewidth=0.5, edgecolor='black', zorder=0)
        ax_1.add_feature(stt_prv, linewidth=0.2, edgecolor='black', zorder=5)
        ax_1.add_feature(roads,linewidth=.4, edgecolor="black")

        
        
        plt.savefig('figs/temperatura/temperatura_{}.png'.format(i))
        #Para que no se muestren las imágenes
        plt.close(fig)

    with imageio.get_writer('figs/temperatura.gif', mode='I') as writer:
            for i in range(0,56):
                for j in range(0,3):
                    filename = 'figs/temperatura/temperatura_{}.png'.format(i)
                    image = imageio.imread(filename)
                    writer.append_data(image)    

In [5]:
gif_temperatura_3_doms(['wrfout_d01_2021-05-11_00:00:00',
                        'wrfout_d02_2021-05-11_00:00:00',
                        'wrfout_d03_2021-05-11_00:00:00'])

/home/fernan/miniconda3/envs/ncl_stable/lib/python3.8/site-packages/cartopy/mpl/geoaxes.py:1478: UserWarning: The following kwargs were not used by contour: 'linewidth'
  result = matplotlib.axes.Axes.contour(self, *args, **kwargs)
